# EPyR Tools - EPR Lineshape Analysis Tutorial

This comprehensive tutorial demonstrates the powerful lineshape analysis capabilities in EPyR Tools. We'll explore different lineshape types, their applications in EPR spectroscopy, and practical analysis techniques.

## Learning Objectives
- Understand different EPR lineshape types (Gaussian, Lorentzian, Voigt)
- Learn to use derivatives and phase rotation for enhanced analysis
- Apply lineshape functions to real EPR scenarios
- Master the unified Lineshape class interface

**Prerequisites:** Basic Python knowledge, understanding of EPR spectroscopy concepts

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import warnings
warnings.filterwarnings('ignore')  # Suppress minor warnings for cleaner output

# Configure matplotlib for better plots
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12
plt.rcParams['lines.linewidth'] = 2

print("📦 Libraries imported successfully!")

In [ ]:
# Import EPyR Tools lineshape functions
import epyr
from epyr.lineshapes import (
    Lineshape, gaussian, lorentzian, voigtian, pseudo_voigt, 
    create_gaussian, create_lorentzian, create_voigt, convspec
)

print(f"🧲 EPyR Tools v{epyr.__version__} lineshape module loaded!")
print(f"📊 Available functions: {epyr.lineshapes.__all__}")

## 1. Fundamental Lineshapes in EPR

EPR spectra exhibit different lineshapes depending on the broadening mechanisms:
- **Gaussian**: Inhomogeneous broadening (distribution of local fields)
- **Lorentzian**: Homogeneous broadening (lifetime effects, collisions)
- **Voigt**: Convolution of both mechanisms (realistic case)

Let's start by comparing these fundamental shapes.

In [ ]:
# Create magnetic field axis (typical EPR range)
B = np.linspace(-15, 15, 1000)  # mT
B_center = 0  # mT
width = 6.0   # mT FWHM

# Generate fundamental lineshapes
gaussian_line = gaussian(B, B_center, width)
lorentzian_line = lorentzian(B, B_center, width)
pseudo_voigt_line = pseudo_voigt(B, B_center, width, eta=0.5)  # 50/50 mix

# Create comparison plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Linear scale
ax1.plot(B, gaussian_line, 'b-', label='Gaussian (inhomogeneous)', linewidth=2.5)
ax1.plot(B, lorentzian_line, 'r-', label='Lorentzian (homogeneous)', linewidth=2.5)
ax1.plot(B, pseudo_voigt_line, 'g--', label='Pseudo-Voigt (50/50 mix)', linewidth=2)

ax1.set_xlabel('Magnetic Field (mT)')
ax1.set_ylabel('EPR Signal (normalized)')
ax1.set_title('EPR Lineshape Comparison')
ax1.legend()
ax1.grid(alpha=0.3)

# Log scale to show tail behavior
ax2.semilogy(B, gaussian_line, 'b-', label='Gaussian', linewidth=2.5)
ax2.semilogy(B, lorentzian_line, 'r-', label='Lorentzian', linewidth=2.5)
ax2.semilogy(B, pseudo_voigt_line, 'g--', label='Pseudo-Voigt', linewidth=2)

ax2.set_xlabel('Magnetic Field (mT)')
ax2.set_ylabel('EPR Signal (log scale)')
ax2.set_title('Tail Behavior Analysis')
ax2.legend()
ax2.grid(alpha=0.3)
ax2.set_ylim(1e-6, 1)

plt.tight_layout()
plt.show()

print("🔍 Key observations:")
print("  • Gaussian: Sharp peak, rapid decay (exponential tails)")
print("  • Lorentzian: Broader peak, slow decay (1/x² tails)")
print("  • Pseudo-Voigt: Intermediate behavior")

## 2. The Unified Lineshape Class

EPyR Tools provides a unified `Lineshape` class that offers a consistent interface for all lineshape types. This makes it easy to switch between different models and compare results.

In [ ]:
# Create different lineshape objects
gauss_shape = Lineshape('gaussian', width=5.0)
lorentz_shape = Lineshape('lorentzian', width=5.0)
pv_shape = Lineshape('pseudo_voigt', width=5.0, alpha=0.3)  # 30% Gaussian, 70% Lorentzian

print("📊 Created lineshape objects:")
print(f"  Gaussian: {gauss_shape}")
print(f"  Lorentzian: {lorentz_shape}")
print(f"  Pseudo-Voigt: {pv_shape}")

# Generate data using the unified interface
B_range = np.linspace(-12, 12, 800)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Basic comparison
ax = axes[0, 0]
ax.plot(B_range, gauss_shape(B_range, 0), 'b-', label='Gaussian')
ax.plot(B_range, lorentz_shape(B_range, 0), 'r-', label='Lorentzian')
ax.plot(B_range, pv_shape(B_range, 0), 'g-', label='Pseudo-Voigt (α=0.3)')
ax.set_title('Unified Interface Comparison')
ax.legend()
ax.grid(alpha=0.3)

# Absorption vs Dispersion
ax = axes[0, 1]
gauss_abs = gauss_shape.absorption(B_range, 0)
gauss_disp = gauss_shape.dispersion(B_range, 0)
ax.plot(B_range, gauss_abs, 'b-', label='Absorption')
ax.plot(B_range, gauss_disp, 'r-', label='Dispersion')
ax.set_title('Absorption vs Dispersion')
ax.legend()
ax.grid(alpha=0.3)

# Parameter modification
ax = axes[1, 0]
gauss_narrow = gauss_shape.set_width(3.0)
gauss_wide = gauss_shape.set_width(8.0)
ax.plot(B_range, gauss_shape(B_range, 0), 'b-', label='Original (w=5)')
ax.plot(B_range, gauss_narrow(B_range, 0), 'g-', label='Narrow (w=3)')
ax.plot(B_range, gauss_wide(B_range, 0), 'r-', label='Wide (w=8)')
ax.set_title('Width Modification')
ax.legend()
ax.grid(alpha=0.3)

# Different centers (hyperfine splitting simulation)
ax = axes[1, 1]
centers = [-4, 0, 4]  # Simulate triplet
intensities = [1, 2, 1]  # 1:2:1 ratio
colors = ['blue', 'red', 'green']

for center, intensity, color in zip(centers, intensities, colors):
    y = intensity * gauss_shape(B_range, center)
    ax.plot(B_range, y, color=color, alpha=0.7, linestyle='--', 
            label=f'Center = {center} mT')

# Total spectrum
total = sum(intensity * gauss_shape(B_range, center) 
           for center, intensity in zip(centers, intensities))
ax.plot(B_range, total, 'k-', linewidth=3, label='Total spectrum')

ax.set_title('Hyperfine Splitting Simulation')
ax.set_xlabel('Magnetic Field (mT)')
ax.set_ylabel('EPR Signal')
ax.legend()
ax.grid(alpha=0.3)

# Add common xlabel and ylabel
for ax in axes[:-1, :].flat:
    ax.set_xlabel('Magnetic Field (mT)')
for ax in axes[:, 0].flat:
    ax.set_ylabel('EPR Signal')

plt.tight_layout()
plt.show()

## 3. Derivative Spectroscopy

EPR spectroscopy commonly uses derivative detection to enhance resolution and reduce noise. Let's explore how derivatives affect different lineshapes.

In [ ]:
# Generate derivatives for both Gaussian and Lorentzian
B_deriv = np.linspace(-10, 10, 800)
width_deriv = 4.0

# Gaussian derivatives
gauss_0 = gaussian(B_deriv, 0, width_deriv, derivative=0)
gauss_1 = gaussian(B_deriv, 0, width_deriv, derivative=1)
gauss_2 = gaussian(B_deriv, 0, width_deriv, derivative=2)

# Lorentzian derivatives
lorentz_0 = lorentzian(B_deriv, 0, width_deriv, derivative=0)
lorentz_1 = lorentzian(B_deriv, 0, width_deriv, derivative=1)
lorentz_2 = lorentzian(B_deriv, 0, width_deriv, derivative=2)

fig, axes = plt.subplots(3, 2, figsize=(14, 12))

# Function (0th derivative)
axes[0, 0].plot(B_deriv, gauss_0, 'b-', linewidth=2.5)
axes[0, 0].set_title('Gaussian Function')
axes[0, 0].set_ylabel('Amplitude')
axes[0, 0].grid(alpha=0.3)

axes[0, 1].plot(B_deriv, lorentz_0, 'r-', linewidth=2.5)
axes[0, 1].set_title('Lorentzian Function')
axes[0, 1].grid(alpha=0.3)

# First derivative
axes[1, 0].plot(B_deriv, gauss_1, 'b-', linewidth=2.5)
axes[1, 0].set_title('Gaussian 1st Derivative')
axes[1, 0].set_ylabel('1st Derivative')
axes[1, 0].axhline(0, color='k', linestyle=':', alpha=0.5)
axes[1, 0].grid(alpha=0.3)

axes[1, 1].plot(B_deriv, lorentz_1, 'r-', linewidth=2.5)
axes[1, 1].set_title('Lorentzian 1st Derivative')
axes[1, 1].axhline(0, color='k', linestyle=':', alpha=0.5)
axes[1, 1].grid(alpha=0.3)

# Second derivative
axes[2, 0].plot(B_deriv, gauss_2, 'b-', linewidth=2.5)
axes[2, 0].set_title('Gaussian 2nd Derivative')
axes[2, 0].set_xlabel('Magnetic Field (mT)')
axes[2, 0].set_ylabel('2nd Derivative')
axes[2, 0].axhline(0, color='k', linestyle=':', alpha=0.5)
axes[2, 0].grid(alpha=0.3)

axes[2, 1].plot(B_deriv, lorentz_2, 'r-', linewidth=2.5)
axes[2, 1].set_title('Lorentzian 2nd Derivative')
axes[2, 1].set_xlabel('Magnetic Field (mT)')
axes[2, 1].axhline(0, color='k', linestyle=':', alpha=0.5)
axes[2, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("📈 Derivative Applications in EPR:")
print("  • 1st derivative: Enhanced resolution, zero-crossing at peak center")
print("  • 2nd derivative: Peak identification, separation of overlapping signals")
print("  • Different shapes show different derivative patterns")

## 4. Phase Rotation: Absorption vs Dispersion

EPR signals can have both absorption and dispersion components. Phase rotation allows us to optimize the signal for different analysis purposes.

In [ ]:
# Demonstrate phase rotation
B_phase = np.linspace(-8, 8, 600)
width_phase = 3.0

# Different phases from 0° to 90°
phases_deg = np.array([0, 15, 30, 45, 60, 75, 90])
phases_rad = np.deg2rad(phases_deg)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

colors = plt.cm.viridis(np.linspace(0, 1, len(phases_deg)))

# Gaussian phase rotation
for i, (phase_deg, phase_rad) in enumerate(zip(phases_deg, phases_rad)):
    y = gaussian(B_phase, 0, width_phase, phase=phase_rad)
    ax1.plot(B_phase, y, color=colors[i], linewidth=2, label=f'{phase_deg}°')

ax1.set_xlabel('Magnetic Field (mT)')
ax1.set_ylabel('EPR Signal')
ax1.set_title('Gaussian Phase Rotation')
ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax1.grid(alpha=0.3)

# Lorentzian phase rotation
for i, (phase_deg, phase_rad) in enumerate(zip(phases_deg, phases_rad)):
    y = lorentzian(B_phase, 0, width_phase, phase=phase_rad)
    ax2.plot(B_phase, y, color=colors[i], linewidth=2, label=f'{phase_deg}°')

ax2.set_xlabel('Magnetic Field (mT)')
ax2.set_ylabel('EPR Signal')
ax2.set_title('Lorentzian Phase Rotation')
ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Show absorption and dispersion separately
gauss_abs, gauss_disp = gaussian(B_phase, 0, width_phase, return_both=True)
lorentz_abs, lorentz_disp = lorentzian(B_phase, 0, width_phase, return_both=True)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Gaussian components
axes[0, 0].plot(B_phase, gauss_abs, 'b-', linewidth=2.5, label='Absorption')
axes[0, 0].plot(B_phase, gauss_disp, 'r-', linewidth=2.5, label='Dispersion')
axes[0, 0].set_title('Gaussian: Absorption vs Dispersion')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# Lorentzian components
axes[0, 1].plot(B_phase, lorentz_abs, 'b-', linewidth=2.5, label='Absorption')
axes[0, 1].plot(B_phase, lorentz_disp, 'r-', linewidth=2.5, label='Dispersion')
axes[0, 1].set_title('Lorentzian: Absorption vs Dispersion')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# Complex representation (2D plot)
axes[1, 0].plot(gauss_abs, gauss_disp, 'b-', linewidth=2.5)
axes[1, 0].set_xlabel('Absorption')
axes[1, 0].set_ylabel('Dispersion')
axes[1, 0].set_title('Gaussian: Complex Plane')
axes[1, 0].grid(alpha=0.3)
axes[1, 0].axis('equal')

axes[1, 1].plot(lorentz_abs, lorentz_disp, 'r-', linewidth=2.5)
axes[1, 1].set_xlabel('Absorption')
axes[1, 1].set_ylabel('Dispersion')
axes[1, 1].set_title('Lorentzian: Complex Plane')
axes[1, 1].grid(alpha=0.3)
axes[1, 1].axis('equal')

plt.tight_layout()
plt.show()

print("🌊 Phase Rotation Properties:")
print("  • 0°: Pure absorption (symmetric, maximum signal)")
print("  • 90°: Pure dispersion (antisymmetric, good for calibration)")
print("  • Mixed phases: Used to correct instrumental phase errors")

## 5. Practical EPR Applications

Let's explore some realistic EPR scenarios where different lineshapes and analysis techniques are crucial.

### 5.1 Multi-component Spectrum Analysis

In [ ]:
# Simulate a multi-component EPR spectrum
B_multi = np.linspace(320, 350, 1000)  # Typical X-band range

# Component 1: Organic radical (narrow Gaussian)
radical1_center = 334.8
radical1_width = 0.8
radical1_intensity = 1.0
radical1 = radical1_intensity * gaussian(B_multi, radical1_center, radical1_width)

# Component 2: Metal complex (broader Lorentzian)
metal_center = 332.5
metal_width = 2.5
metal_intensity = 0.7
metal = metal_intensity * lorentzian(B_multi, metal_center, metal_width)

# Component 3: Solid-state defect (pseudo-Voigt)
defect_center = 340.2
defect_width = 3.0
defect_intensity = 0.9
defect = defect_intensity * pseudo_voigt(B_multi, defect_center, defect_width, eta=0.3)

# Total spectrum
total_spectrum = radical1 + metal + defect

# Add realistic noise
np.random.seed(42)  # For reproducibility
noise_level = 0.02
noise = noise_level * np.random.normal(size=len(B_multi))
noisy_spectrum = total_spectrum + noise

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Individual components
ax1.plot(B_multi, radical1, 'b--', alpha=0.7, label='Organic radical (Gaussian)')
ax1.plot(B_multi, metal, 'r--', alpha=0.7, label='Metal complex (Lorentzian)')
ax1.plot(B_multi, defect, 'g--', alpha=0.7, label='Solid defect (Pseudo-Voigt)')
ax1.plot(B_multi, total_spectrum, 'k-', linewidth=2.5, label='Total spectrum')

ax1.set_ylabel('EPR Signal (a.u.)')
ax1.set_title('Multi-component EPR Spectrum Analysis')
ax1.legend()
ax1.grid(alpha=0.3)

# With noise (realistic experimental data)
ax2.plot(B_multi, noisy_spectrum, 'k-', alpha=0.8, linewidth=1, label='Experimental data')
ax2.plot(B_multi, total_spectrum, 'r-', linewidth=2, label='Fitted model')

ax2.set_xlabel('Magnetic Field (mT)')
ax2.set_ylabel('EPR Signal (a.u.)')
ax2.set_title('Realistic EPR Data with Noise')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("🔬 Multi-component Analysis Results:")
print(f"  Component 1 (Organic): g ≈ {2.00232 * 334.8/334.8:.5f}, Width = {radical1_width:.1f} mT")
print(f"  Component 2 (Metal): g ≈ {2.00232 * 334.8/332.5:.5f}, Width = {metal_width:.1f} mT")
print(f"  Component 3 (Defect): g ≈ {2.00232 * 334.8/340.2:.5f}, Width = {defect_width:.1f} mT")

### 5.2 Temperature-Dependent Broadening

In [ ]:
# Simulate temperature-dependent EPR broadening
B_temp = np.linspace(-8, 8, 600)
temperatures = [77, 150, 200, 273, 300, 400, 500]  # Kelvin
base_width = 1.0  # mT at room temperature
T_ref = 300  # Reference temperature

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

colors = plt.cm.coolwarm(np.linspace(0, 1, len(temperatures)))
widths = []

for i, T in enumerate(temperatures):
    # Temperature broadening model: width ∝ √T (simple case)
    width_T = base_width * np.sqrt(T / T_ref)
    widths.append(width_T)
    
    # Generate Lorentzian (dominant at higher T due to spin-phonon coupling)
    y = lorentzian(B_temp, 0, width_T)
    ax1.plot(B_temp, y, color=colors[i], linewidth=2, label=f'{T} K')

ax1.set_xlabel('Magnetic Field (mT)')
ax1.set_ylabel('EPR Signal')
ax1.set_title('Temperature-Dependent Broadening')
ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax1.grid(alpha=0.3)

# Plot width vs temperature
ax2.plot(temperatures, widths, 'ro-', linewidth=2, markersize=6)
ax2.set_xlabel('Temperature (K)')
ax2.set_ylabel('Linewidth (mT)')
ax2.set_title('Linewidth vs Temperature')
ax2.grid(alpha=0.3)

# Fit and show the relationship
T_fit = np.linspace(77, 500, 100)
width_fit = base_width * np.sqrt(T_fit / T_ref)
ax2.plot(T_fit, width_fit, 'b--', alpha=0.7, label='√T fit')
ax2.legend()

plt.tight_layout()
plt.show()

print("🌡️ Temperature Effects on EPR Linewidth:")
print("  • Low T: Narrow lines, long spin relaxation times")
print("  • High T: Broad lines, fast spin-phonon coupling")
print("  • Relationship often follows power laws (√T, T, T²)")

### 5.3 Derivative Spectroscopy for Enhanced Resolution

In [ ]:
# Demonstrate the power of derivative spectroscopy
B_deriv = np.linspace(-10, 10, 800)

# Two overlapping Gaussian signals
signal1_center = -1.5
signal2_center = 1.5
width = 3.0
intensity1 = 1.0
intensity2 = 0.8

# Individual signals
signal1_abs = intensity1 * gaussian(B_deriv, signal1_center, width)
signal2_abs = intensity2 * gaussian(B_deriv, signal2_center, width)
overlapped_abs = signal1_abs + signal2_abs

# First derivatives
signal1_d1 = intensity1 * gaussian(B_deriv, signal1_center, width, derivative=1)
signal2_d1 = intensity2 * gaussian(B_deriv, signal2_center, width, derivative=1)
overlapped_d1 = signal1_d1 + signal2_d1

# Second derivatives
signal1_d2 = intensity1 * gaussian(B_deriv, signal1_center, width, derivative=2)
signal2_d2 = intensity2 * gaussian(B_deriv, signal2_center, width, derivative=2)
overlapped_d2 = signal1_d2 + signal2_d2

fig, axes = plt.subplots(3, 1, figsize=(12, 12))

# Absorption (0th derivative)
axes[0].plot(B_deriv, signal1_abs, 'b--', alpha=0.6, label='Signal 1')
axes[0].plot(B_deriv, signal2_abs, 'r--', alpha=0.6, label='Signal 2')
axes[0].plot(B_deriv, overlapped_abs, 'k-', linewidth=2.5, label='Total')
axes[0].set_ylabel('Absorption Signal')
axes[0].set_title('Overlapping EPR Signals: Resolution Enhancement')
axes[0].legend()
axes[0].grid(alpha=0.3)

# First derivative
axes[1].plot(B_deriv, signal1_d1, 'b--', alpha=0.6, label='Signal 1 (1st deriv)')
axes[1].plot(B_deriv, signal2_d1, 'r--', alpha=0.6, label='Signal 2 (1st deriv)')
axes[1].plot(B_deriv, overlapped_d1, 'k-', linewidth=2.5, label='Total (1st deriv)')
axes[1].axhline(0, color='gray', linestyle=':', alpha=0.5)
axes[1].set_ylabel('1st Derivative')
axes[1].legend()
axes[1].grid(alpha=0.3)

# Second derivative
axes[2].plot(B_deriv, signal1_d2, 'b--', alpha=0.6, label='Signal 1 (2nd deriv)')
axes[2].plot(B_deriv, signal2_d2, 'r--', alpha=0.6, label='Signal 2 (2nd deriv)')
axes[2].plot(B_deriv, overlapped_d2, 'k-', linewidth=2.5, label='Total (2nd deriv)')
axes[2].axhline(0, color='gray', linestyle=':', alpha=0.5)
axes[2].set_xlabel('Magnetic Field (mT)')
axes[2].set_ylabel('2nd Derivative')
axes[2].legend()
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Derivative Spectroscopy Benefits:")
print("  • 1st derivative: Zero crossings mark peak centers")
print("  • 2nd derivative: Peak maxima become more distinct")
print("  • Enhanced resolution for overlapping signals")
print("  • Reduced baseline effects")

## 6. Spectrum Convolution and Broadening

The convolution function allows us to apply broadening to stick spectra or simulate instrumental effects.

In [ ]:
# Create a stick spectrum (e.g., from quantum chemical calculations)
B_stick = np.linspace(-15, 15, 1000)
stick_spectrum = np.zeros_like(B_stick)

# Add peaks at calculated positions with computed intensities
peak_positions = [-8, -3, 1, 5, 9]  # mT
peak_intensities = [0.8, 1.2, 2.0, 1.0, 0.6]  # Relative intensities

for pos, intensity in zip(peak_positions, peak_intensities):
    idx = np.argmin(np.abs(B_stick - pos))
    stick_spectrum[idx] = intensity

# Apply different types of broadening
dx = B_stick[1] - B_stick[0]
broadening_width = 2.0

# Pure Gaussian broadening (α=1)
gaussian_broad = convspec(stick_spectrum, dx, width=broadening_width, alpha=1.0)

# Pure Lorentzian broadening (α=0)
lorentzian_broad = convspec(stick_spectrum, dx, width=broadening_width, alpha=0.0)

# Mixed broadening (α=0.3 → 30% Gaussian, 70% Lorentzian)
mixed_broad = convspec(stick_spectrum, dx, width=broadening_width, alpha=0.3)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Original stick spectrum
ax1.stem(B_stick, stick_spectrum, linefmt='k-', markerfmt='ko', basefmt=' ')
ax1.set_ylabel('Intensity')
ax1.set_title('Original Stick Spectrum (Calculated Transitions)')
ax1.grid(alpha=0.3)
ax1.set_xlim(B_stick[0], B_stick[-1])

# Broadened spectra
ax2.plot(B_stick, gaussian_broad, 'b-', linewidth=2.5, label='Gaussian broadening')
ax2.plot(B_stick, lorentzian_broad, 'r-', linewidth=2.5, label='Lorentzian broadening')
ax2.plot(B_stick, mixed_broad, 'g--', linewidth=2, label='Mixed broadening (30/70)')

ax2.set_xlabel('Magnetic Field (mT)')
ax2.set_ylabel('EPR Signal')
ax2.set_title('Convolution Results')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("📡 Convolution Applications:")
print("  • Convert calculated stick spectra to realistic lineshapes")
print("  • Model instrumental broadening effects")
print("  • Simulate temperature-dependent broadening")
print("  • Test different broadening mechanisms")

## 7. Advanced Analysis: g-Factor and Hyperfine Coupling

Let's use lineshape analysis for practical EPR parameter determination.

In [ ]:
# Simulate hyperfine splitting patterns
def simulate_hyperfine_pattern(B_range, g_factor, A_coupling, nuclear_spin, microwave_freq=9.5e9):
    """
    Simulate EPR spectrum with hyperfine coupling
    
    Parameters:
    - B_range: Magnetic field range (mT)
    - g_factor: Electronic g-factor
    - A_coupling: Hyperfine coupling constant (mT)
    - nuclear_spin: Nuclear spin quantum number
    - microwave_freq: Microwave frequency (Hz)
    """
    # Physical constants
    mu_B = 9.274e-24  # Bohr magneton (J/T)
    h = 6.626e-34     # Planck constant (J·s)
    g_e = 2.00232     # Free electron g-factor
    
    # Calculate resonance field
    B_res = (h * microwave_freq) / (mu_B * g_factor) * 1000  # Convert to mT
    
    # Generate hyperfine lines
    m_I_values = np.arange(-nuclear_spin, nuclear_spin + 1)
    spectrum = np.zeros_like(B_range)
    
    for m_I in m_I_values:
        # Field position for this hyperfine line
        B_line = B_res + m_I * A_coupling
        
        # Intensity (assume equal for simplicity)
        intensity = 1.0
        
        # Add Lorentzian lineshape
        line_width = 0.8  # mT
        line_shape = intensity * lorentzian(B_range, B_line, line_width)
        spectrum += line_shape
    
    return spectrum, B_res

# Simulate different radical systems
B_hyperfine = np.linspace(330, 340, 1000)

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Hydrogen atom (I = 1/2, doublet)
ax = axes[0, 0]
spectrum_H, B_res_H = simulate_hyperfine_pattern(B_hyperfine, 2.0023, 0.5, 0.5)
ax.plot(B_hyperfine, spectrum_H, 'b-', linewidth=2)
ax.set_title('Hydrogen Atom (I = 1/2)\nDoublet Pattern')
ax.set_ylabel('EPR Signal')
ax.grid(alpha=0.3)
ax.text(0.02, 0.98, f'g = 2.0023\nA = 0.5 mT', transform=ax.transAxes, 
        verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

# 2. Nitrogen radical (I = 1, triplet)
ax = axes[0, 1]
spectrum_N, B_res_N = simulate_hyperfine_pattern(B_hyperfine, 2.0055, 1.0, 1)
ax.plot(B_hyperfine, spectrum_N, 'r-', linewidth=2)
ax.set_title('Nitrogen Radical (I = 1)\nTriplet Pattern')
ax.grid(alpha=0.3)
ax.text(0.02, 0.98, f'g = 2.0055\nA = 1.0 mT', transform=ax.transAxes, 
        verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

# 3. Multi-nuclear system (two equivalent protons)
ax = axes[1, 0]
# For two equivalent I=1/2 nuclei: intensity ratio 1:2:1
B_center = 335
A_proton = 0.8
positions = [B_center - A_proton, B_center, B_center + A_proton]
intensities = [1, 2, 1]  # 1:2:1 ratio

spectrum_2H = np.zeros_like(B_hyperfine)
for pos, intensity in zip(positions, intensities):
    line = intensity * lorentzian(B_hyperfine, pos, 0.6)
    spectrum_2H += line

ax.plot(B_hyperfine, spectrum_2H, 'g-', linewidth=2)
ax.set_title('Two Equivalent Protons\n1:2:1 Triplet')
ax.set_xlabel('Magnetic Field (mT)')
ax.set_ylabel('EPR Signal')
ax.grid(alpha=0.3)
ax.text(0.02, 0.98, f'g = 2.0023\nA = 0.8 mT\n2 × H', transform=ax.transAxes, 
        verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

# 4. g-factor analysis
ax = axes[1, 1]
g_values = [1.995, 2.0023, 2.010, 2.030]
colors = ['blue', 'red', 'green', 'purple']

for g_val, color in zip(g_values, colors):
    spectrum_g, _ = simulate_hyperfine_pattern(B_hyperfine, g_val, 0, 0)
    ax.plot(B_hyperfine, spectrum_g, color=color, linewidth=2, label=f'g = {g_val}')

ax.set_title('g-Factor Effects on Resonance Field')
ax.set_xlabel('Magnetic Field (mT)')
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("🧬 Hyperfine and g-Factor Analysis:")
print("  • g-factor: Determines resonance field position")
print("  • Hyperfine coupling: Creates splitting patterns")
print("  • Pattern multiplicity: 2nI + 1 (n = number of equivalent nuclei)")
print("  • Intensity ratios: Given by binomial coefficients")

## 8. Summary and Best Practices

This tutorial has covered the comprehensive lineshape analysis capabilities in EPyR Tools. Here are the key takeaways:

In [ ]:
# Summary plot: All lineshape types together
B_summary = np.linspace(-12, 12, 800)
width_summary = 4.0

# Generate all major lineshape types
shapes = {
    'Gaussian': gaussian(B_summary, 0, width_summary),
    'Lorentzian': lorentzian(B_summary, 0, width_summary),
    'Pseudo-Voigt (η=0.3)': pseudo_voigt(B_summary, 0, width_summary, eta=0.3),
    'Pseudo-Voigt (η=0.7)': pseudo_voigt(B_summary, 0, width_summary, eta=0.7),
}

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

colors = ['blue', 'red', 'green', 'orange']
linestyles = ['-', '-', '--', ':']

# Linear scale
for (name, y), color, ls in zip(shapes.items(), colors, linestyles):
    ax1.plot(B_summary, y, color=color, linestyle=ls, linewidth=2.5, label=name)

ax1.set_xlabel('Magnetic Field (mT)')
ax1.set_ylabel('EPR Signal (normalized)')
ax1.set_title('EPyR Tools Lineshape Comparison')
ax1.legend()
ax1.grid(alpha=0.3)

# Log scale for tail comparison
for (name, y), color, ls in zip(shapes.items(), colors, linestyles):
    ax2.semilogy(B_summary, np.abs(y) + 1e-8, color=color, linestyle=ls, linewidth=2.5, label=name)

ax2.set_xlabel('Magnetic Field (mT)')
ax2.set_ylabel('|EPR Signal| (log scale)')
ax2.set_title('Tail Behavior (Log Scale)')
ax2.legend()
ax2.grid(alpha=0.3)
ax2.set_ylim(1e-6, 1)

plt.tight_layout()
plt.show()

print("📚 EPyR Tools Lineshape Analysis - Key Features:")
print("\n🔧 Available Functions:")
print("  • gaussian() - Pure Gaussian profiles")
print("  • lorentzian() - Pure Lorentzian profiles")
print("  • voigtian() - True Voigt profiles (convolution)")
print("  • pseudo_voigt() - Fast pseudo-Voigt approximation")
print("  • lshape() - General mixed profiles")
print("  • convspec() - Spectrum convolution")
print("  • Lineshape class - Unified interface")

print("\n📊 Key Capabilities:")
print("  • Derivatives (0th, 1st, 2nd order)")
print("  • Phase rotation (absorption ↔ dispersion)")
print("  • Flexible width parameters")
print("  • Normalization options")
print("  • High numerical accuracy")

print("\n🧲 EPR Applications:")
print("  • Multi-component spectral fitting")
print("  • g-factor and hyperfine analysis")
print("  • Temperature-dependent studies")
print("  • Derivative spectroscopy")
print("  • Instrumental broadening simulation")

print("\n💡 Best Practices:")
print("  • Use Gaussian for inhomogeneous broadening")
print("  • Use Lorentzian for homogeneous broadening")
print("  • Use Voigt/pseudo-Voigt for realistic spectra")
print("  • Apply derivatives for enhanced resolution")
print("  • Check phase for optimal signal")
print("  • Validate with known standards")

print("\n✅ Tutorial Complete!")
print("You're now ready to use EPyR Tools for advanced EPR lineshape analysis.")